In [1]:
!wget -O AnyLengthBertModelAndUtils.py https://raw.githubusercontent.com/DominicJW/AnyLengthBert/main/AnyLengthBertModelAndUtils.py
!wget -O AnyLengthBert.pt https://www.dropbox.com/scl/fi/u1m76jo7ai2vd5kbk1kym/AnyLengthBert.pt?rlkey=qhdw3f5yqw8qzmfslbz795qqx&st=wixhnsh2&dl=1
!wget -O dev.csv https://drive.usercontent.google.com/u/0/uc?id=1Rh6ZgNk6svFvlXokAnX2OeY7zw9ghXlH&export=download
!wget -O train.csv https://drive.usercontent.google.com/download?id=1B6h9GBUoiWV00FHWWv4XL7OKWSv2peTR&export=download
!pip install datasets

drive_path = "./"

--2024-04-23 01:49:56--  https://raw.githubusercontent.com/DominicJW/AnyLengthBert/main/AnyLengthBertModelAndUtils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12727 (12K) [text/plain]
Saving to: ‘AnyLengthBertModelAndUtils.py’

AnyLengthBertModelA 100%[===================>]  12.43K  --.-KB/s    in 0s      

2024-04-23 01:49:56 (71.1 MB/s) - ‘AnyLengthBertModelAndUtils.py’ saved [12727/12727]

--2024-04-23 01:49:56--  https://www.dropbox.com/scl/fi/u1m76jo7ai2vd5kbk1kym/AnyLengthBert.pt?rlkey=qhdw3f5yqw8qzmfslbz795qqx
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https:/

In [3]:
from AnyLengthBertModelAndUtils import my_collate, CustomDataset, set_seed, evaluate_loss, SentencePairClassifier, test_prediction
from  AnyLengthBertModelAndUtils import MaskedGlobalAvgPool1d, MaskedGlobalMaxPool1d
import torch
from torch.utils.data import DataLoader
import pandas as pd
import os

In [4]:
test =  pd.read_csv(os.path.join(drive_path,"dev.csv"))
df_test = pd.DataFrame(test)

In [5]:
print("Reading test data...")
from plotly import figure_factory as ff
bert_model = 'bert-base-uncased'
maxlen = 128
bs = 32
test_set = CustomDataset(df_test,bert_model,maxlen)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
path_to_output_file = "my_results"
test_loader = DataLoader(test_set, batch_size=bs, num_workers=5,collate_fn=my_collate)
model = SentencePairClassifier(bert_model=bert_model,freeze_bert=False)
state_dict = torch.load('AnyLengthBert.pt')

model = state_dict
model.eval()
print("Predicting on test data...")
test_prediction(net=model, device=device, dataloader=test_loader, with_labels=True,  # set the with_labels parameter to False if your want to get predictions on a dataset without labels
                result_file=path_to_output_file)
print()
print("Predictions are available in : {}".format(path_to_output_file))
labels_test = df_test['label']  # true labels
probs_test = pd.read_csv(path_to_output_file, header=None)[0]  # prediction probabilities
threshold = 0.5   # you can adjust this threshold for your own dataset
preds_test=(probs_test>=threshold).astype('uint8') # predicted labels using the above fixed threshold

from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef,accuracy_score, confusion_matrix

all_labels = labels_test
all_predictions = preds_test

precision = precision_score(all_labels, all_predictions)
recall = recall_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions)
mcc = matthews_corrcoef(all_labels, all_predictions)
accuracy = accuracy_score(all_labels,all_predictions)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"MCC: {mcc}")
print(f"Accuracy: {accuracy}")


cm = confusion_matrix(all_labels, all_predictions)


# Labels for the axes
labels = ['Negative', 'Positive']

# Create annotated heatmap
fig = ff.create_annotated_heatmap(cm, x=labels, y=labels, colorscale='Blues', showscale=True)
fig.update_layout(title='Confusion Matrix with Plotly',
                  xaxis_title='Predicted Value',
                  yaxis_title='Actual Value',
                  yaxis_autorange='reversed')  # Reverse y-axis to have the layout similar to scikit-learn's confusion matrix plot

# Add custom xaxis and yaxis titles
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].font.size = 12

# Show the plot
fig.show()


Reading test data...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Predicting on test data...


  0%|          | 0/186 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 186/186 [00:17<00:00, 10.59it/s]



Predictions are available in : my_results
Precision: 0.7592014971927635
Recall: 0.7611006879299562
F1 Score: 0.760149906308557
MCC: 0.6713649524457147
Accuracy: 0.8704016199797503
